In [219]:
%load_ext autoreload
%autoreload 2
%cd C:\MAD4AG
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
C:\MAD4AG


In [220]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import geopandas

import warnings

warnings.filterwarnings('ignore')

In [221]:
file_name = f'./dbs/intermediate/stops_1_new.parquet'

#Read each batchs file
df = pd.read_parquet(file_name)
df = df[df.holiday_s != 1]
df = df[df.weekday_s == 1]
df = df.drop(['holiday_s', 'weekday_s'], axis=1)
df.drop_duplicates(subset=['uid', 'cluster'], keep='first', inplace=True)

In [103]:
df_h = pd.read_parquet(f'./dbs/intermediate/home_inference.parquet')
df_h.drop_duplicates(subset='uid', keep='first', inplace=True)
df_h['home_potential'] = 1

df_w = pd.read_parquet(f'./dbs/intermediate/work_inference.parquet')
df_w.drop_duplicates(subset='uid', keep='first', inplace=True)

df_1 = pd.merge(df, df_h[['uid', 'cluster', 'home_potential']], on=['uid', 'cluster'], how="left")

df_1 = pd.merge(df_1, df_w[['uid', 'cluster', 'work_potential']], on=['uid', 'cluster'], how="left")

df_1.fillna(value={"home_potential": 0, "work_potential": 0}, inplace=True)

df_1[['home_potential', 'work_potential']] = df_1[['home_potential', 'work_potential']].astype(int)

In [222]:
ppl_having_home = list(df_1['uid'][df_1.home_potential == 1].unique())
df_1 = df_1[df_1.uid.isin(ppl_having_home)]
df_2 = df_1[(df_1.home_potential == 1) | (df_1.work_potential == 1)]

In [135]:
gdf_2 = geopandas.GeoDataFrame(df_2, geometry=geopandas.points_from_xy(df_2.cluster_lng, df_2.cluster_lat), crs="EPSG:4326")

gdf_2.drop("Deso", axis=1, inplace=True)

DeSO = geopandas.read_file(
    f'C:/Synthetic_population_new/caglar/synthetic_sweden/input/deso_statistik_shp/Bef_Alder_region.shp')

DeSO.loc[:, 'adult_pop'] = DeSO.loc[:, [ 'Ald20_24', 'Ald25_29', 'Ald30_34', 'Ald35_39', 'Ald40_44', 'Ald45_49', 'Ald50_54', 'Ald55_59', 'Ald60_64', 'Ald65_69', 'Ald70_74', 'Ald75_79', 'Ald80_w']].sum(axis=1) + DeSO.loc[:,'Ald16_19']/2

DeSO_arb = geopandas.read_file(
    f'C:/Synthetic_population_new/caglar/synthetic_sweden/input/deso_statistik_shp/Bef_Sysselsattning_region.shp')

DeSO = pd.merge(DeSO, DeSO_arb[['Deso','Forvarb']], left_on=['Deso'], right_on=['Deso'], how="left")
DeSO.rename(columns={'Forvarb':'employee_pop'}, inplace=True )

print(DeSO.crs)

DeSO.to_crs(4326, inplace=True)
print(DeSO.crs)

PROJCS["SWEREF99 TM",GEOGCS["SWEREF99",DATUM["SWEREF99",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6619"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","3006"]]
epsg:4326


In [136]:
DeSO['employee_pop'].sum()/ DeSO['adult_pop'].sum()

0.566448852284483

In [137]:
gdf_2 = gdf_2.sjoin(DeSO[['Deso','geometry']], how="left")


In [138]:
gdf_2_home = gdf_2[gdf_2.home_potential== 1]

gdf_2_home= gdf_2_home.groupby('Deso')['uid'].count().reset_index().rename(columns={"uid":'home_count'})

In [139]:
gdf_2_home.home_count.sum()

278832

In [122]:
employees = gdf_2['uid'][gdf_2.work_potential== 1].unique()

gdf_2_work = gdf_2[gdf_2.uid.isin(employees)]


In [123]:
gdf_2_work= gdf_2_work[gdf_2.home_potential== 1].groupby('Deso')['uid'].count().reset_index().rename(columns={"uid":'employee_count'})



In [205]:
gdf_2_stat = pd.merge(DeSO[['Deso','adult_pop', 'employee_pop']], gdf_2_home, left_on=['Deso'], right_on=['Deso'], how="left")
gdf_2_stat = pd.merge(gdf_2_stat, gdf_2_work, left_on=['Deso'], right_on=['Deso'], how="left")

In [124]:

gdf_2_stat['employee_count']=gdf_2_stat['employee_count'].fillna(0).astype(int)

In [125]:
#https://github.com/MobiSegInsights/mobi-social-segregation-se/blob/main/src/MobiSegInsightsSE.etl.1.0/12-home-detection-and-filtering.ipynb

In [126]:
gdf_2_stat.columns

Index(['Deso', 'adult_pop', 'employee_pop', 'home_count', 'employee_count'], dtype='object')

In [127]:
gdf_2_stat.loc[:, 'wt_p'] = gdf_2_stat.loc[:, 'adult_pop'] / gdf_2_stat.loc[:, 'home_count']

#gdf_2_stat['wt_p']=gdf_2_stat['wt_p'].fillna(0)



In [128]:
w0 = ((np.std(gdf_2_stat.loc[:, 'wt_p']) / np.mean(gdf_2_stat.loc[:, 'wt_p'])) ** 2 + 1) ** 0.5 * 3.5 * np.median(gdf_2_stat.loc[:, 'wt_p'])


In [130]:
gdf_2_stat.loc[gdf_2_stat['wt_p'] > w0, 'wt_p'] = w0

np.sum(gdf_2_stat['wt_p']*gdf_2_stat['home_count'])

8057544.306518847

In [131]:
gdf_2_stat.adult_pop.sum()

8060839.0

In [132]:
df_2 = pd.merge(df_2, gdf_2_stat, left_on=['Deso'], right_on=['Deso'], how="left")


In [140]:
df_2['wt_p'][df_2.home_potential==1].sum()

8060022.994413406

In [210]:
aaa = df_2[df_2.home_potential==1]

aaa['work_potential'][aaa.uid.isin(employees)]=1


In [211]:
aaa = aaa[['uid', 'Deso', 'work_potential', 'adult_pop', 'employee_pop', 'home_count', 'employee_count', 'wt_p']].sort_values(by=['Deso'])

In [214]:
def ipf_wtp(data):
    for it in range(5):
        employees = data['wt_p'][data['work_potential']==1].sum()
        data['wt_p'][data['work_potential']==1] = data['wt_p'][data['work_potential']==1] * (data['employee_pop']/ employees)
        population = data['wt_p'].sum()
        data['wt_p'] = data['wt_p'] * (data['adult_pop']/ population)
    return data

In [215]:

tqdm.pandas()
aaa = aaa.groupby('Deso').progress_apply(ipf_wtp)

  0%|          | 0/5985 [00:00<?, ?it/s]

In [197]:
9783*0.05837

571.0337099999999

In [206]:
bbb = aaa[aaa.Deso== '0160C1180']

In [207]:
bbb['wt_p'].sum()

958.0

In [200]:
bbb['wt_p'][bbb.work_potential==1].sum()


606.9511941252065